In [2]:
import h5py
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt

#import persUF      as pUF
import sklearn_tda as tda

from sklearn.preprocessing            import *
from sklearn.svm                      import *
from sklearn.ensemble                 import *
from sklearn.decomposition            import *
from sklearn.linear_model             import *
from sklearn.model_selection          import *
from sklearn.pipeline                 import *
from sklearn.manifold                 import *
from sklearn.neighbors                import *
from sklearn.metrics                  import *
from sklearn.metrics.pairwise         import *
from sklearn.feature_selection        import *
from sklearn.kernel_ridge             import *

ImportError: No module named 'sklearn_tda'

In [ ]:
D = pUF.geodesic_diagrams_on_shape(b"61.off", [0,1])

In [ ]:
plt.scatter(np.array(D[0][1])[:,1], np.array(D[0][1])[:,0])
plt.axis([0.0,2.0,0.0,2.0])
plt.show()

In [ ]:
def diag_to_array(data):
    dataset, num_diag = [], len(data["0"].keys())
    for dim in data.keys():
        X = []
        for diag in range(num_diag):
            pers_diag = np.array(data[dim][str(diag)])
            X.append(pers_diag)
        dataset.append(X)
    return dataset

def diag_to_dict(D):
    X = dict()
    for f in D.keys():
        df = diag_to_array(D[f])
        for dim in range(len(df)):
            X[str(dim) + "_" + f] = df[dim]
    return X

In [ ]:
train_lab  = pd.read_csv("train.csv")
train_diag = diag_to_dict(h5py.File("train_diag.hdf5", "r"))

In [ ]:
test_size                   = 0.2
train_num_pts               = train_lab.shape[0]    
perm                        = np.random.permutation(train_num_pts)
limit                       = np.int(test_size * train_num_pts)
test_sub, train_sub         = perm[:limit], perm[limit:]
train_num_pts, test_num_pts = len(train_sub), len(test_sub)

train_full_labels           = train_lab["part"]
le                          = LabelEncoder()
train_labels                = np.array(le.fit_transform(train_full_labels[train_sub]))
test_labels                 = np.array(le.transform(train_full_labels[test_sub]))

train_full_diag             = train_diag["1_geodesic"]
train_diag                  = [train_full_diag[i] for i in train_sub]
test_diag                   = [train_full_diag[i] for i in test_sub]
 
print(train_num_pts, test_num_pts)

In [ ]:
pipe = Pipeline([("Separator",     tda.FiniteSelector(limit=np.inf)),
                 ("Rotator",       tda.DiagramPreprocessor(scaler=tda.BirthPersistenceTransform())),
                 ("TDA",           tda.PersistenceImage()),
                 ("Estimator",     SVC())
                 ])

param =    [{"Rotator__use":            [True],
             "TDA":                     [tda.PersistenceImage()], 
             "TDA__im_resolution":      [ [20,20] ],
             "TDA__kernel":             [ gauss(s) for s in [0.1, 1.0] ],
             "Estimator":               [SVC()],
             "Estimator__C":            [1,10,100],
             "Estimator__gamma":        [0.001,0.0001]},
            
            {"Rotator__use":            [False],
             "TDA":                     [tda.Landscape()], 
             "TDA__resolution":         [100,1000],
             "Estimator":               [SVC()],
             "Estimator__C":            [1,10,100],
             "Estimator__gamma":        [0.001,0.0001]},
           
            {"Rotator__use":            [False],
             "TDA":                     [tda.SlicedWasserstein()], 
             "TDA__gaussian_bandwidth": [0.01, 0.1, 1.0, 10.0],
             "Estimator":               [SVC(kernel="precomputed")],
             "Estimator__C":            [1,10,100]}]

model = GridSearchCV(pipe, param)

In [ ]:
model.fit(train_diag, train_labels)
print("Train accuracy = " + str(model.score(train_diag, train_labels)))
print("Test accuracy  = " + str(model.score(test_diag,  test_labels)))